# Homework: Competitive Grammar Writing

## Development of S1.gr

To develop S1.gr which would be used to generate grammars, the base output that we would want from _S1.gr_ would be to output sentences that are potentially difficult for other teams to parse. To do this, it was decided that a 
custom grammar should be written from scratch to generate these. As a custom grammar would not be able to be developed rigidly in little time, it would likely be able to generate weird sentences.

Using this approach in mind, S1.gr was developed completely custom by hand. To develop _S1.gr_, it was decided that **example_sentences.txt** would be the baseline used when developing _S1.gr_. 

When developing _S1.gr_, the main approach used was a recursive approach, in which every "appropriate" sentence would be defined as a **_Phrase_** and everything else would be a bunch of **_Phrase_** constructs recursively joined together. 

In example, lets take a look at a sentence from **example_sentences.txt**:

> Arthur will have been riding for eight nights .

Over here, the phrase,

> riding for eight nights .

Is a valid sentence and could very well be generated to be passed off as a regular sentence. As such, it should register as a **_Phrase_** construct. Moving on, the beginning of the sentence,

> Arthur will have been

Is also a valid phrase in some contexts of English. As a result, it will also be registered as a **_Phrase_** construct. Due to this, a construct that manually joins two **_Phrase_** constructs was also made, which is called a **_ComplexPhrase_** construct.

_Note: all "constructs" are essentially Non-Terminal rules._

### Handling Conjunctions

Due to already taking the approach of wanting to recursively define each valid part of a sentence as a construct in order to recursively parse a sentence, the next question would be what of conjunctions?

Let us consider another sentence from **example_sentences.txt**:

> Arthur rode and drank from his chalice .

Following the previous logic laid out, one might expect to parse 

> Arthur rode

and 

> drank from his chalice

as **_Phrase_** constructs joined by a conjunction. However, doing so would give problems as conjunctions can be used arbitrarily many times in a sentence. As a result, with our current approach, we would end up having to define _arbitrarily many layers of_ **_ComplexPhrase_** _constructs in order to parse such sentences._ As a result, we take a different approach, which is by making a rule/construct that takes in a form of _(conjunction) (Phrase)_. 

This rule is defined as **_APhrase_** in _S1.gr_. What we essentially do is that we expand a **_Phrase_** to be able to take in a subphrase and any sort of **_APhrase_**. As a result, we will be able to parse arbitrarily many conjunctions as possible as they would all just be registered as **_APhrase_** constructs.

### How about the rest of grammar constructs that are too minor to be phrases?

The rest of the constructs were made as per sentences in **example_sentences.txt**. As we based the custom grammar off the file, we made all the necessary constructs into _Non-Terminals_ that **_Phrase_** would consume in some form or manner. 

Examples of these would be constructs like 

> ...from his...

> ...for eight nights...

and so on. These are all tinier frequently-used constructs in the form of grammar rules like _(Preposition)(Pronoun)_ etc which are made as necessary as they appear in sentences from **example_sentences.txt**. As we are basing our custom grammar to be made off such a small amount of sentences, naturally some relations would not exist when they should, but that is of no matter, as the primary goal of _S1.gr_ is to be used to generate sentences that would be potentially difficult for other teams to parse.

## The Development of S2.gr (parsing grammars)

The grammars in S2 are auto-generated by training on devset.trees, marked example-sentences.txt, and some other resources all together.

See following the code to read in trees, transform them to CNF, and extract grammars out.

In [5]:
from nltk import tree

class Tree:
    _trees = []
    _root = None

    def __init__(self, sentence):
        while sentence.find("(") != -1:
            left_bracket_index = sentence.find("(")
            right_bracket_index = left_bracket_index
            number_of_open_brackets = 1
            for (i, each_char) in enumerate(sentence[left_bracket_index + 1:]):
                if each_char == '(':
                    number_of_open_brackets += 1
                elif each_char == ')':
                    number_of_open_brackets -= 1
                    if number_of_open_brackets == 0:
                        right_bracket_index = i + 1
                        break
            self._root = self.analyze_grammars(sentence[left_bracket_index: left_bracket_index + right_bracket_index + 1], True)
            self._trees.append(self.analyze_grammars(sentence[left_bracket_index: left_bracket_index + right_bracket_index + 1], True))
            sentence = sentence[left_bracket_index + right_bracket_index + 1:]

    def transformToCNF(self, sentence):
        t = tree.Tree.fromstring(sentence, remove_empty_top_bracketing=True)

        # convert the tree to CNF
        t.chomsky_normal_form()
        return str(t)

    # should return a tree's root node
    def analyze_grammars(self, sentence, top):
        #print(sentence)
        sentence = self.transformToCNF(sentence)
        #print("after CNF transform")
        #print(sentence)
        if len(sentence.split()) == 2:
            # this is a terminal grammar segment
            terminal_parts = sentence.replace("(", " ").replace(")", " ").split()
            if not terminal_parts:
                return None
            if terminal_parts[0] == terminal_parts[1]:
                terminal_parts[0] *= 2
            new_node = GrammarNode(terminal_parts[0], terminal_parts[1])
            new_node.set_is_terminal(True)
            return new_node

        # remove first ( and last )
        sentence_without_out_most_bracket = sentence[sentence.find("(") + 1: sentence.rfind(")")]
        # split it
        # first token is left hand side
        left_side = sentence_without_out_most_bracket.split()[0]
        # slice the next grammar by first ( and first ) until no more (
        right_side = []
        temp_sentence = sentence_without_out_most_bracket
        while temp_sentence.find("(") != -1:
            left_bracket_index = temp_sentence.find("(")
            right_bracket_index = left_bracket_index
            number_of_open_brackets = 1
            for (i, each_char) in enumerate(temp_sentence[left_bracket_index + 1:]):
                if each_char == '(':
                    number_of_open_brackets += 1
                elif each_char == ')':
                    number_of_open_brackets -= 1
                    if number_of_open_brackets == 0:
                        right_bracket_index = i + 1
                        break
            to_append = self.analyze_grammars(temp_sentence[left_bracket_index: left_bracket_index + right_bracket_index + 1],False)
            if to_append is not None:
                right_side.append(to_append)
            temp_sentence = temp_sentence[left_bracket_index + right_bracket_index + 1:]

        new_node = GrammarNode(left_side, right_side)
        if top:
            new_node = GrammarNode("S1", [new_node])
        return new_node

    def dump_to_grammars(self):
        grammars = {}
        for each_tree in self._trees:
            self.dump_to_grammars_internal(each_tree, grammars)
        return grammars

    def dump_to_grammars_internal(self, node, grammars):
        if node.is_terminal():
            left = node.get_left_hand_side()
            right = node.get_right_hand_side()
            a_grammar = Grammar(left, right)
            if grammars.get(a_grammar) is None:
                #uncommenting these two lines will remove terminal grammars from the generated grammar file
                grammars[a_grammar] = 1
                grammars[a_grammar] = 0  #this line
            else:
                grammars[a_grammar] += 1
                grammars[a_grammar] = 0  #this line
            return

        left = node.get_left_hand_side()
        right = " ".join(map(lambda n: n.get_left_hand_side(), node.get_right_hand_side()))
        a_grammar = Grammar(left, right)
        if grammars.get(a_grammar) is None:
            grammars[a_grammar] = 1
            #uncommenting these two lines will remove non-terminal grammars from the generated file
            #grammars[a_grammar] = 0      #this line
        else:
            grammars[a_grammar] += 1
            #grammars[a_grammar] = 0       #this line
        for each in node.get_right_hand_side():
            self.dump_to_grammars_internal(each, grammars)




class GrammarNode:
    _left_hand_side = None
    _right_hand_side = []
    _isTerminal = False

    def __init__(self, left_hand_side=None, right_hand_side=[]):
        self._left_hand_side = left_hand_side
        self._right_hand_side = right_hand_side

    def set_left_hand_side(self, left_hand_side):
        self._left_hand_side = left_hand_side

    def set_right_hand_side(self, right_hand_side):
        self._right_hand_side = right_hand_side

    def set_is_terminal(self, is_terminal):
        self._isTerminal = is_terminal

    def get_left_hand_side(self):
        return self._left_hand_side

    def get_right_hand_side(self):
        return self._right_hand_side

    def is_terminal(self):
        return self._isTerminal

    def attach_node(self, node):
        self._children.push(node)

class Grammar:
    _left = ""
    _right = ""
    _prob = -1

    def __init__(self, left, right):
        self._left = left
        self._right = right

    def __hash__(self):
        return hash((self._left, self._right))

    def __eq__(self, other):
        return other is not None and self._left == other._left and self._right == other._right

    def __str__(self):
        return self._left + "  " + self._right


with open("devset.trees", "r") as f:
    sentences = ''.join(f.readlines())
f.close()

stuff = Tree(sentences)
grammars_with_count = stuff.dump_to_grammars()

left_set = set()
for key in grammars_with_count.keys():
    left_set.add(key._left)

for each_left in left_set:
    filtered_keys = {k:v for (k,v) in grammars_with_count.items() if k._left == each_left}
    total_count = 0
    for each in filtered_keys:
        total_count += grammars_with_count.get(each)
    # for grammar_to_update in filtered_keys:
    #     grammar_to_update._prob = grammars_with_count.get(grammar_to_update) / float(total_count)

    
##generated file will have extracted grammars
##see comments in method dump_to_grammars_internal() to see how to change types of grammars to be generated
with open("NT.gr", "w") as wf:
    for item in grammars_with_count.items():
        if item[1] is not 0:
            wf.write(str(item[1]) + " " + str(item[0]) + "\n")
wf.close()

with open("T.gr", "r") as wf:
    for line in wf.readlines():
        print(line)
wf.close()


1 NNP  Whoa

9 EX  there

159 .  !

3 NN  Halt

10 WP  Who

1 VBZ  goes

4 RB  there

90 .  ?

11 PRP  It

37 VBZ  is

78 PRP  I

233 ,,  ,

18 NNP  Arthur

1 NN  son

60 IN  of

1 NNP  Uther

1 NNP  Pendragon

11 IN  from

97 DT  the

8 NN  castle

11 NNP  Camelot

207 ..  .

10 NNP  King

8 NNP  Britons

1 NN  defeator

1 NNP  Saxons

1 NN  sovereign

12 DT  all

2 NNP  England

1 VB  Pull

2 JJ  other

12 CD  one

10 VBP  am

13 CC  And

17 DT  this

16 PRP$  my

1 NN  trusty

1 NN  servant

3 NNP  Patsy

15 PRP  We

12 VBP  have

2 VBN  ridden

1 NN  length

42 CC  and

1 NN  breadth

4 NN  land

36 IN  in

2 NN  search

7 NNS  knights

9 WP  who

6 MD  will

7 VB  join

15 PRP  me

2 NN  court

2 MD  must

1 VB  speak

8 IN  with

27 PRP$  your

3 NN  lord

4 NN  master

25 WP  What

1 VBP  ridden

12 IN  on

78 DT  a

1 NN  horse

9 UH  Yes

3 NNP  You're

1 VBG  using

2 NNS  coconuts

1 NNP  You've

9 VBD  got

3 CD  two

1 JJ  empty

1 NNS  halves

6 NN  coconut

76 PRP  you



In [6]:
with open("NT.gr", "r") as wf:
    for line in wf.readlines():
        print(line)
wf.close()

26 S1  INTJ

1 INTJ  NNP INTJ|<ADVP-.>

1 INTJ|<ADVP-.>  ADVP .

2 ADVP  EX

74 S1  NP

18 NP  NN .

37 S1  SBARQ

19 SBARQ  WHNP SBARQ|<SQ-.>

44 WHNP  WP

31 SBARQ|<SQ-.>  SQ .

8 SQ  VP

4 VP  VBZ ADVP

93 ADVP  RB

275 S1  S

89 S  NP S|<VP-.>

324 NP  PRP

113 S|<VP-.>  VP ..

2 VP  VBZ VP|<NP-PP>

27 VP|<NP-PP>  NP PP

1 NP  NP NP|<,-NP-,-NP-,>

1 NP|<,-NP-,-NP-,>  ,, NP|<NP-,-NP-,>

1 NP|<NP-,-NP-,>  NP NP|<,-NP-,>

80 NP  NNP

2 NP|<,-NP-,>  ,, NP|<NP-,>

2 NP|<NP-,>  NP ,,

61 NP  NP PP

54 NP  NN

173 PP  IN NP

14 NP  NNP NNP

102 NP  DT NN

1 NP  NP NP|<,-NP-,-ADJP-.>

25 NP  DT NNP

1 NP|<,-NP-,-ADJP-.>  ,, NP|<NP-,-ADJP-.>

1 NP|<NP-,-ADJP-.>  NP NP|<,-ADJP-.>

1 NP|<,-ADJP-.>  ,, NP|<ADJP-.>

5 NP|<ADJP-.>  ADJP .

4 ADJP  NN PP

33 S  VP .

34 VP  VB NP

2 NP  DT NP|<JJ-CD>

2 NP|<JJ-CD>  JJ CD

14 VP  VBP

55 S1  FRAG

1 FRAG  CC FRAG|<NP-NP-.>

1 FRAG|<NP-NP-.>  NP FRAG|<NP-.>

30 NP  DT

10 FRAG|<NP-.>  NP ..

1 NP  PRP$ NP|<NN-NN-NNP>

1 NP|<NN-NN-NNP>  NN NP|<NN-NN